In this project I'll explore the file "Superstore" using different Python libraries, like Pandas and Seaborn. I'll create data visualizations to find patterns and insights in the data.

In [27]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../data/superstore.xlsm')
data.loc[0:5,['Segment', 'Country','City',]]

,Segment,Country,City
0,Consumer,United States,Henderson
1,Consumer,United States,Henderson
2,Corporate,United States,Los Angeles
3,Consumer,United States,Fort Lauderdale
4,Consumer,United States,Fort Lauderdale
5,Consumer,United States,Los Angeles


In [9]:
data[~(data['Country'] == 'United States')]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit


There is no orders outside the United States.
There are non-null values in any column

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [29]:
data.describe(include=[np.number])

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


Testing apply function

In [24]:
data_sample = data.loc[0:5,['Segment', 'Country','City','Sales','Discount','Profit']]
data_sample['Profit_2'] = data_sample.apply(lambda x: x['Profit'] if x['City'] == 'Henderson' else 0, axis = 1)
data_sample

,Segment,Country,City,Sales,Discount,Profit,Profit_2
0,Consumer,United States,Henderson,261.9600,0.00,41.9136,41.9136
1,Consumer,United States,Henderson,731.9400,0.00,219.5820,219.5820
2,Corporate,United States,Los Angeles,14.6200,0.00,6.8714,0.0000
3,Consumer,United States,Fort Lauderdale,957.5775,0.45,-383.0310,0.0000
4,Consumer,United States,Fort Lauderdale,22.3680,0.20,2.5164,0.0000
5,Consumer,United States,Los Angeles,48.8600,0.00,14.1694,0.0000


Using groupby function

In [6]:
data.groupby('Ship Mode')['Ship Mode'].count().sort_values(ascending=False)

Ship Mode
First Class       1538
Same Day           543
Second Class      1945
Standard Class    5968
Name: Ship Mode, dtype: int64

In [23]:
data.groupby('Segment')['Segment'].count()sort_values(ascending=False)

SyntaxError: invalid syntax (1426445677.py, line 1)

In [22]:
data.groupby(['Category','Sub-Category'])['Category'].count().sort_values(ascending=False)

Category         Sub-Category
Office Supplies  Binders         1523
                 Paper           1370
Furniture        Furnishings      957
Technology       Phones           889
Office Supplies  Storage          846
                 Art              796
Technology       Accessories      775
Furniture        Chairs           617
Office Supplies  Appliances       466
                 Labels           364
Furniture        Tables           319
Office Supplies  Envelopes        254
Furniture        Bookcases        228
Office Supplies  Fasteners        217
                 Supplies         190
Technology       Machines         115
                 Copiers           68
Name: Category, dtype: int64

In [11]:
data.groupby(['Category'])['Category'].count().sort_values(ascending=False)

Category
Furniture          2121
Office Supplies    6026
Technology         1847
Name: Category, dtype: int64

In [21]:
data.groupby(['Region','State','City'])['Region'].count().sort_values(ascending=False)

Region  State         City         
East    New York      New York City    915
West    California    Los Angeles      747
East    Pennsylvania  Philadelphia     537
West    California    San Francisco    510
        Washington    Seattle          428
                                      ... 
South   Florida       Kissimmee          1
                      Jupiter            1
        Arkansas      Springdale         1
                      Rogers             1
West    Wyoming       Cheyenne           1
Name: Region, Length: 604, dtype: int64

In [17]:
len(data['City'].unique())

531